In [164]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%load_ext autoreload
%autoreload 2

import pandas
from mino_repo.mino_repo_class import MinoRepo
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [204]:
zfile = 'dades/ml-latest-small/links.csv'
links = pandas.read_csv(zfile)

zfile = 'dades/ml-latest-small/movies.csv'
movies = pandas.read_csv(zfile)

zfile = 'dades/ml-latest-small/ratings.csv'
ratings = pandas.read_csv(zfile)

zfile = 'dades/ml-latest-small/tags.csv'
tags = pandas.read_csv(zfile)


print(links.columns.tolist())
print(movies.columns.tolist())
print(ratings.columns.tolist())
print(tags.columns.tolist())

pd = pandas.merge(links, movies, how='outer')
pd = pandas.merge(pd, ratings, how='outer')
mr = MinoRepo(pd)
mr.summary

['movieId', 'imdbId', 'tmdbId']
['movieId', 'title', 'genres']
['userId', 'movieId', 'rating', 'timestamp']
['userId', 'movieId', 'tag', 'timestamp']
*fact_columns:  
	movieId , imdbId , tmdbId , title , genres , userId , rating , timestamp
	rows: 100063
*dim_fields:  
	
*foreign Keys:  
*master_tables_names:  
	


In [205]:
mr.create_dim(['movieId', 'title', 'genres'], 'dim_movie')
mr.create_dim(['imdbId', 'tmdbId'], 'dim_movie_2')
mr.summary

*fact_columns:  
	userId , rating , timestamp
	rows: 100004
*dim_fields:  
	movieId , title , genres , imdbId , tmdbId
*foreign Keys:  
	__FK__001 -> dim_movie
	__FK__002 -> dim_movie_2
*master_tables_names:  
	dim_movie ; dim_movie_2


In [211]:
mr.filter_facts('Toy Story (1995)','title', negative = False).head()

,userId,rating,timestamp
0,7.0,3.0,8.518667e+08
1,9.0,4.0,9.386292e+08
2,13.0,5.0,1.331380e+09
3,15.0,2.0,9.979383e+08
4,19.0,3.0,8.551901e+08


In [223]:
df = mr.dim_fields

In [224]:
l = [x for x in df.items()]

In [280]:
dim_fields_to_load = ['title','genres','imdbId']
tables = set([df[x] for x in dim_fields_to_load])
d = {}
for i in tables:
    d[i] = [x for x in dim_fields_to_load if df[x] == i]
    
t = mr._fact_table.iloc[0:4]
for table_name in d.keys():
    table = mr.master_tables[table_name]
    fields_table = d[table_name]
    t = t.join(table[fields_table], on=table.index.name)

In [279]:
mr._fact_table.iloc[0:4]


,userId,rating,timestamp,__FK__001,__FK__002
0,7.0,3.0,8.518667e+08,0,0
1,9.0,4.0,9.386292e+08,0,0
2,13.0,5.0,1.331380e+09,0,0
3,15.0,2.0,9.979383e+08,0,0


In [272]:
mr.master_tables['dim_movie'].index.name

'__FK__001'

In [273]:
t = mr._fact_table.iloc[0:4]
dim_table = mr.master_tables['dim_movie']
dim_fields = ['title']
t = t.join(dim_table[dim_fields], on=dim_table.index.name)
dim_table = mr.master_tables['dim_movie_2']
dim_fields = ['imdbId']
t = t.join(dim_table[dim_fields], on=dim_table.index.name)

list_to_drop=

In [281]:
t

,userId,rating,timestamp,__FK__001,__FK__002,imdbId,title,genres
0,7.0,3.0,8.518667e+08,0,0,114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,9.0,4.0,9.386292e+08,0,0,114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,13.0,5.0,1.331380e+09,0,0,114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15.0,2.0,9.979383e+08,0,0,114709,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [269]:
mr.full_thread('dim_movie')

In [261]:
pd.iloc[0:4]

,movieId,imdbId,tmdbId,title,genres,userId,rating,timestamp
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,3.0,8.518667e+08
1,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9.0,4.0,9.386292e+08
2,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,5.0,1.331380e+09
3,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.0,9.979383e+08
